# Stream test  
hdfs 에 구매 요청 정보가 들어오는 경우, stream 방식으로 요청을 수신하고 처리  

In [63]:
# 필요 라이브러리 임포트  
import socket
import sys
import os
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
from os.path import abspath
import findspark
import time 

# 환경변수 정의  
scale = 1000 # 1000 만 건 수준
PRJ_ROOT = '/user/root'
APP_NAME = '04-stream'
DB_NAME = 'inven'

# 데이터의 파일 포맷 및 파일명  
tbl_name = 'inven/request'
file_format = 'csv'

# 스파크 생성 
def spark_creation():
    spark = SparkSession.builder.master('yarn').appName(APP_NAME)\
    .config('spark.driver.cores', '2').config('spark.driver.memory', '4g')\
    .config('spark.num.executors', '2')\
    .config('spark.executor.cores', '2').config('spark.executor.memory', '4g')\
    .config('spark.jars', '/hive-bin/lib/mysql-connector-java-5.1.49-bin.jar')\
    .config('spark.driver.extraClassPath', '/hive-bin/lib/mysql-connector-java-5.1.49-bin.jar').getOrCreate()
    sc = spark.sparkContext
    sc
    return spark

def display_output(outTarget, sqlString, interval = 1):
    from IPython.display import display, clear_output
    from pyspark.sql.streaming import StreamingQuery 
    import time 
    if isinstance(outTarget, StreamingQuery):
        while True:
            clear_output(wait=False)
            #print(f"Query : {sqlString}")
            display(spark.sql(sqlString).show())
            print(f"outTarget : {outTarget}")
            # print(f"temp : {(temp[0])}")
            #display(outTarget.select("type").show())
            time.sleep(interval)
    else:
        print("Not instance......")
        spark.sql(sqlString).show()
        
def define_schema():
    from pyspark.sql.types import StructType, StructField, StringType, LongType
    columns = [
        StructField("type", StringType())
        , StructField("qty", LongType())
    ]
    inven_schema = StructType(columns)
    return inven_schema

In [49]:
%%time
spark = spark_creation()

CPU times: user 20.5 ms, sys: 2.17 ms, total: 22.6 ms
Wall time: 15.1 s


/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [66]:
# Stream 요청 처리  
from pyspark.sql.functions import explode, split
temp = ['']
dataSchema = define_schema() 
#lines = spark.readStream.format("hdfs").option("", "").load()
lines = spark.readStream.schema(dataSchema).csv(f"{PRJ_ROOT}/{tbl_name}")

lines.writeStream.option("checkpointLocation", f"{PRJ_ROOT}/{tbl_name}").toTable("myTable")
spark.read.table("myTable").show()

+----+---+
|type|qty|
+----+---+
+----+---+



In [ ]:
query_name = "lines9"
# lines.createOrReplaceTempView(query_name)
sql = f"select * from myTable "
# df = spark.sql(sql)
#outQ = df.writeStream.queryName(query_name).format("memory").trigger(processingTime="30 seconds").outputMode("append").start()
outQ = lines.writeStream.queryName(query_name).trigger(processingTime="5 seconds").outputMode("append").format("memory").start()  


display_output(outQ, sql)
# display_output(lines, sql)

+----+---+
|type|qty|
+----+---+
+----+---+



None

outTarget : <pyspark.sql.streaming.StreamingQuery object at 0x7fa3c91ea518>


In [47]:
spark.stop()